# 텍스트 전처리
- 전처리 기법 : 토큰화, 노이즈/불용어 제거, 통일화, 품사태깅, 벡터화
- 위의 전처리 기법이 실제 분석에서 어떻게 이루어지는지 파악하고 익혀보자

## konlpy 설치

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 32.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import json
import re

from tqdm.notebook import tqdm

In [3]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

In [4]:
import os
from datetime import datetime

## Download Dataset(github에서 NSMC 데이터셋 다운로드)

In [5]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [6]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [7]:
train.head(15)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


# EDA

다양하게 데이터를 탐색해 봅시다.

- 데이터 수는 어떤지?
- 데이터 라벨의 분포는 어떤지?
- NA 데이터는 없는지?
- 중복된 데이터는 없는지?
- 등등

## 위의 데이터를 적절하게 전처리 해 봅시다.

 토큰화 + 노이즈/불용어제거 + 통일화 + 주요 품사 추출(품사태깅)

### 토큰화

문장을 단어로 나누어 봅시다.

### 노이즈 제거

re 패키지 안에 sub라는 함수는 글자를 찾아서 대체해 주는 역할을 합니다.

In [9]:
text = '포스코ICT는 25일 경기도 판교 사옥에서 청소년들이 개발한 인공지능(AI) 기반 애플리케이션을 직접 선보이는‘2022 AI 유스 챌린지(Youth Challenge)’를 개최했다.'
replaced_text = re.sub('ICT', 'DX', text)
print(replaced_text)
replaced_text = re.sub('\d+', '', text)
print(replaced_text)

포스코DX는 25일 경기도 판교 사옥에서 청소년들이 개발한 인공지능(AI) 기반 애플리케이션을 직접 선보이는‘2022 AI 유스 챌린지(Youth Challenge)’를 개최했다.
포스코ICT는 일 경기도 판교 사옥에서 청소년들이 개발한 인공지능(AI) 기반 애플리케이션을 직접 선보이는‘ AI 유스 챌린지(Youth Challenge)’를 개최했다.


In [ ]:
# 위에 re.sub를 사용해 한글만 남기고 나머지 문자를 제거해 봅시다
??
result

In [ ]:
# 1글자짜리가 과연 필요할까요? 한 글자로 된 단어를 제거합시다.
??
result

['더빙', '진짜', '짜증나네요', '목소리']

### 품사태깅 + 어간추출

In [ ]:
okt = Okt()
word_review = okt.pos(review_text, stem=True)
word_review

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나다', 'Adjective'),
 ('목소리', 'Noun')]

### 불용어 제거

### 전처리 함수 구현

위에서 여러분이 했던 전처리 과정을 하나의 연속된 하나의 과정으로 만들어 볼까요?

한 문장이 들어가서 클리닝 된 리스트로 return 받는 함수를 작성해봅시다.

In [ ]:
def preprocessing(review):
    ??
    return word_list

In [ ]:
for x in X_train[:100]:
    print(preprocessing(x))

['더빙', '진짜', '짜증나다']
['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍다', '않다']
['무재', '밓었', '다그', '래서', '보다', '추천']
['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']
['몬페', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '늙다', '보이다', '하다', '커스틴', '던스트', '이쁘다', '보이다']
['걸음', '떼다', '초등학교', '학년', '생인', '살다', '영화', '반개', '아깝다']
['원작', '긴장감', '제대로', '살리다', '하다']
['반개', '아깝다', '나오다', '이응경', '길용우', '생활', '정말', '해도', '그것', '낫다', '납치', '감금', '반복', '반복', '드라마', '가족', '없다', '연기', '하다', '사람']
['액션', '없다', '재미', '있다', '안되다', '영화']
['왜케', '평점', '낮다', '헐리우드', '화려하다', '길들이다', '있다']
['인피니트', '진짜']
['볼때', '눈물나다', '죽다', '년대', '향수', '허진호', '감성', '절제', '멜로', '달인']
['울면', '횡단보도', '건너다', '뛰다', '치다', '이범수', '연기', '드럽다', '하다']
['담백하다', '깔끔하다', '좋다', '로만', '보다', '보다', '자꾸', '잊어버리다', '사람', '이다']
['취향', '존중', '다지', '진짜', '내생', '극장', '보다', '영화', '가장', '노잼', '감동', '스토리', '어거지', '감동', '어거지']
['매번', '긴장', '되다', '재밋음']
['차다', '사람', '웃기다', '바스코', '이기', '락스', '끄다', '바비', '이기', '아이돌', '깔다', '다그', '끄다', '안달', '보이다']
['굿바이', '레닌', '표절',